In [ ]:
import pandas as pd

In [ ]:
file_name = "citaschallenge(1).xlsx"
df = pd.read_excel(file_name)

In [ ]:
df.head()

,GENERO,EDAD,ESPECIALIDAD,TIPO_AFILIACION,FECHA_CITA,id,ESTAFINAL
0,FEMENINO,41.0,ORTODONCIA,GOLD,2013-07-02 06:30:00,9373,3
1,FEMENINO,49.0,TERAPIA OCUPACIONAL,SILVER,2013-07-02 06:30:00,62884,2
2,FEMENINO,41.0,FISIOTERAPIA,GOLD,2013-07-02 06:30:00,72568,2
3,FEMENINO,49.0,FISIOTERAPIA,GOLD,2013-07-02 06:50:00,5733,1
4,FEMENINO,36.0,RADIOLOGIA,GOLD,2013-07-02 06:50:00,22397,3


In [ ]:
# Encontrar el número de valores nulos por cada columna
nulos_por_columna = df.isnull().sum()

# Mostrar los valores nulos por columna
print("\nNúmero de valores nulos por columna:")
print(nulos_por_columna)


Número de valores nulos por columna:
GENERO             0
EDAD               0
ESPECIALIDAD       0
TIPO_AFILIACION    0
FECHA_CITA         0
id                 0
ESTAFINAL          0
dtype: int64


In [ ]:
df.dtypes

GENERO                     object
EDAD                      float64
ESPECIALIDAD               object
TIPO_AFILIACION            object
FECHA_CITA         datetime64[ns]
id                          int64
ESTAFINAL                   int64
dtype: object

# Modificaciones dataset para modelamiento

In [ ]:
# Convertir la columna FECHA_CITA a formato datetime
df['FECHA_CITA'] = pd.to_datetime(df['FECHA_CITA'])

# Crear características adicionales basadas en la fecha
df['DIA_SEMANA'] = df['FECHA_CITA'].dt.dayofweek  # Día de la semana (0=Monday, 6=Sunday)
df['HORA_CITA'] = df['FECHA_CITA'].dt.hour        # Hora del día

# Mostrar las primeras filas del dataframe para verificar
print("Primeras filas del dataframe con nuevas características:")
print(df.head())

Primeras filas del dataframe con nuevas características:
     GENERO  EDAD         ESPECIALIDAD TIPO_AFILIACION          FECHA_CITA  \
0  FEMENINO  41.0           ORTODONCIA            GOLD 2013-07-02 06:30:00   
1  FEMENINO  49.0  TERAPIA OCUPACIONAL          SILVER 2013-07-02 06:30:00   
2  FEMENINO  41.0         FISIOTERAPIA            GOLD 2013-07-02 06:30:00   
3  FEMENINO  49.0         FISIOTERAPIA            GOLD 2013-07-02 06:50:00   
4  FEMENINO  36.0           RADIOLOGIA            GOLD 2013-07-02 06:50:00   

      id  ESTAFINAL  DIA_SEMANA  HORA_CITA  
0   9373          3           1          6  
1  62884          2           1          6  
2  72568          2           1          6  
3   5733          1           1          6  
4  22397          3           1          6  


In [ ]:
# Codificación de variables categóricas usando pd.get_dummies()
df = pd.get_dummies(df, columns=['GENERO', 'ESPECIALIDAD', 'TIPO_AFILIACION'], drop_first=True)

# Mostrar las primeras filas del dataframe para verificar
print("Primeras filas del dataframe con variables categóricas codificadas:")
print(df.head())

# Explicación:
# 1. Utilizamos `pd.get_dummies()` para convertir las variables categóricas en variables dummy (indicadoras). Esto es necesario porque los modelos de aprendizaje automático requieren que todas las entradas sean numéricas.
# 2. La opción `drop_first=True` se utiliza para evitar la multicolinealidad eliminando una de las categorías (la primera) en cada variable categórica.

Primeras filas del dataframe con variables categóricas codificadas:
   EDAD          FECHA_CITA     id  ESTAFINAL  DIA_SEMANA  HORA_CITA  \
0  41.0 2013-07-02 06:30:00   9373          3           1          6   
1  49.0 2013-07-02 06:30:00  62884          2           1          6   
2  41.0 2013-07-02 06:30:00  72568          2           1          6   
3  49.0 2013-07-02 06:50:00   5733          1           1          6   
4  36.0 2013-07-02 06:50:00  22397          3           1          6   

   GENERO_MASCULINO  ESPECIALIDAD_CARDIOLOGIA  \
0             False                     False   
1             False                     False   
2             False                     False   
3             False                     False   
4             False                     False   

   ESPECIALIDAD_CIRUGIA DE LA MANO  ESPECIALIDAD_CIRUGIA GENERAL  ...  \
0                            False                         False  ...   
1                            False                        

# Division train-test

In [ ]:
# Seleccionar características y etiqueta
X = df.drop(columns=['id', 'FECHA_CITA', 'ESTAFINAL'])
y = df['ESTAFINAL']

# Dividir el conjunto de datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar las dimensiones de los conjuntos de entrenamiento y prueba
print(f"Dimensiones de X_train: {X_train.shape}")
print(f"Dimensiones de X_test: {X_test.shape}")
print(f"Dimensiones de y_train: {y_train.shape}")
print(f"Dimensiones de y_test: {y_test.shape}")

# Explicación:
# 1. Eliminamos las columnas 'ID' y 'FECHA_CITA' porque no son relevantes para la predicción del estado final de las citas.
# 2. La columna 'ESTAFINAL' es nuestra etiqueta objetivo, por lo que la separamos del resto del dataframe.
# 3. Dividimos el conjunto de datos en conjuntos de entrenamiento y prueba utilizando `train_test_split()` con una proporción de 80-20. El parámetro `random_state` asegura que la división sea reproducible.

Dimensiones de X_train: (54120, 37)
Dimensiones de X_test: (13530, 37)
Dimensiones de y_train: (54120,)
Dimensiones de y_test: (13530,)


# Modelo No balanceados

In [ ]:
# Función para calcular el F score ponderado
def calcular_f_ponderado(y_true, y_pred):
    f1_incomplida = f1_score(y_true, y_pred, labels=[3], average='macro')
    f1_cancelada = f1_score(y_true, y_pred, labels=[1], average='macro')
    f1_cumplida = f1_score(y_true, y_pred, labels=[2], average='macro')

    f_ponderado = 0.5 * f1_incomplida + 0.3 * f1_cancelada + 0.2 * f1_cumplida
    return f_ponderado

In [ ]:
# Importar las librerías necesarias
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score

In [ ]:
# Ajustar las clases para que comiencen en 0
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1

# Inicializar los modelos
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'XGBoost': xgb.XGBClassifier(eval_metric='mlogloss'),
    'KNN': KNeighborsClassifier()
}

# Función para calcular el F score ponderado
def calcular_f_ponderado(y_true, y_pred):
    f1_incomplida = f1_score(y_true, y_pred, labels=[3], average='macro')
    f1_cancelada = f1_score(y_true, y_pred, labels=[1], average='macro')
    f1_cumplida = f1_score(y_true, y_pred, labels=[2], average='macro')

    f_ponderado = 0.5 * f1_incomplida + 0.3 * f1_cancelada + 0.2 * f1_cumplida
    return f_ponderado

# Inicializar y entrenar los modelos nuevamente
results = {}
for model_name, model in models.items():
    if model_name == 'XGBoost':
        model.fit(X_train, y_train_adjusted)
        y_pred = model.predict(X_test)
        y_pred = y_pred + 1  # Ajustar las predicciones de vuelta a las clases originales
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    f_ponderado = calcular_f_ponderado(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    results[model_name] = {
        'f_ponderado': f_ponderado,
        'classification_report': report
    }

    print(f"Results for {model_name}:")
    print(f"Pondered F Score: {f_ponderado}")
    print(classification_report(y_test, y_pred))
    print("-" * 60)

# Explicación:
# 1. Definimos la función `calcular_f_ponderado` que calcula los F scores individuales para cada clase y combina estos resultados de acuerdo al peso proporcionado en las instrucciones.
# 2. Entrenamos y evaluamos cada modelo nuevamente, ahora calculando y mostrando el F score ponderado para cada uno.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for Logistic Regression:
Pondered F Score: 0.2614301357898002
              precision    recall  f1-score   support

           1       0.67      0.00      0.00      2452
           2       0.66      0.98      0.79      8595
           3       0.50      0.13      0.21      2483

    accuracy                           0.65     13530
   macro avg       0.61      0.37      0.33     13530
weighted avg       0.63      0.65      0.54     13530

------------------------------------------------------------
Results for Random Forest:
Pondered F Score: 0.3760740908146947
              precision    recall  f1-score   support

           1       0.28      0.19      0.22      2452
           2       0.70      0.81      0.75      8595
           3       0.36      0.28      0.32      2483

    accuracy                           0.60     13530
   macro avg       0.45      0.43      0.43     13530
weighted avg       0.56      0.60      0.57     13530

-------------------------------------------

#  Modelos con clases balanceadas

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Mostrar la distribución de clases antes y después del rebalanceo
print("Distribución de clases antes del rebalanceo:", Counter(y_train))
print("Distribución de clases después del rebalanceo:", Counter(y_train_res))

# Explicación:
# 1. Usamos SMOTE para crear ejemplos sintéticos de las clases minoritarias.
# 2. Aplicamos SMOTE al conjunto de entrenamiento para rebalancear las clases.
# 3. Mostramos la distribución de clases antes y después del rebalanceo para verificar el efecto.

Distribución de clases antes del rebalanceo: Counter({2: 34430, 3: 10172, 1: 9518})
Distribución de clases después del rebalanceo: Counter({1: 34430, 2: 34430, 3: 34430})


In [ ]:
# Inicializar los modelos
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'XGBoost': xgb.XGBClassifier(eval_metric='mlogloss', use_label_encoder=False),
    'SVM': KNeighborsClassifier()
}

# Entrenar y evaluar cada modelo con datos rebalanceados
results = {}
for model_name, model in models.items():
    if model_name == 'XGBoost':
        model.fit(X_train_res, y_train_res - 1)
        y_pred = model.predict(X_test)
        y_pred = y_pred + 1  # Ajustar las predicciones de vuelta a las clases originales
    else:
        model.fit(X_train_res, y_train_res)
        y_pred = model.predict(X_test)

    f_ponderado = calcular_f_ponderado(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    results[model_name] = {
        'f_ponderado': f_ponderado,
        'classification_report': report
    }

    print(f"Results for {model_name}:")
    print(f"Pondered F Score: {f_ponderado}")
    print(classification_report(y_test, y_pred))
    print("-" * 60)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for Logistic Regression:
Pondered F Score: 0.37301908056199085
              precision    recall  f1-score   support

           1       0.26      0.15      0.19      2452
           2       0.71      0.78      0.74      8595
           3       0.32      0.35      0.34      2483

    accuracy                           0.59     13530
   macro avg       0.43      0.43      0.42     13530
weighted avg       0.56      0.59      0.57     13530

------------------------------------------------------------
Results for Random Forest:
Pondered F Score: 0.3867132917451759
              precision    recall  f1-score   support

           1       0.25      0.29      0.27      2452
           2       0.71      0.66      0.68      8595
           3       0.32      0.36      0.34      2483

    accuracy                           0.54     13530
   macro avg       0.43      0.43      0.43     13530
weighted avg       0.56      0.54      0.54     13530

------------------------------------------

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Inicializar el modelo base (Árbol de Decisión)
base_model = DecisionTreeClassifier()

# Inicializar el modelo Bagging
bagging_model = BaggingClassifier(base_estimator=base_model, n_estimators=100, random_state=42)

# Entrenar el modelo con los datos rebalanceados
bagging_model.fit(X_train_res, y_train_res)

# Evaluar el modelo
y_pred_bagging = bagging_model.predict(X_test)
f_ponderado_bagging = calcular_f_ponderado(y_test, y_pred_bagging)

print(f"F Score ponderado para Bagging con Árboles de Decisión: {f_ponderado_bagging}")
print(classification_report(y_test, y_pred_bagging))

# Explicación:
# 1. Inicializamos un `DecisionTreeClassifier` como el modelo base.
# 2. Utilizamos `BaggingClassifier` para crear un ensemble de árboles de decisión.
# 3. Entrenamos el modelo Bagging con los datos rebalanceados.
# 4. Evaluamos el modelo y calculamos el F score ponderado.

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


F Score ponderado para Bagging con Árboles de Decisión: 0.3874031961405734
              precision    recall  f1-score   support

           1       0.25      0.28      0.27      2452
           2       0.71      0.66      0.68      8595
           3       0.33      0.36      0.34      2483

    accuracy                           0.54     13530
   macro avg       0.43      0.43      0.43     13530
weighted avg       0.56      0.54      0.55     13530



# Salida de Excel

In [ ]:
# Obtener las predicciones finales (aquí asumimos que el mejor modelo es Random Forest, pero puedes cambiarlo según tu preferencia)
best_model = models['Random Forest']
best_model.fit(X_train, y_train)
y_pred_final = best_model.predict(X_test)

# Crear un DataFrame con los IDs y las predicciones
df_test = X_test.copy()
df_test['ID'] = df.loc[X_test.index, 'id']
df_test['ESTAFINAL'] = y_pred_final

# Crear el DataFrame con las columnas requeridas
df_result = df_test[['ID', 'ESTAFINAL']]

# Guardar el DataFrame en un archivo Excel
output_file = 'predicciones_citas.xlsx'
df_result.to_excel(output_file, index=False)